In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import cv2
import os 
import random
import matplotlib.pyplot as plt
import pickle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image

In [3]:
DIRECTORY = r'/content/drive/MyDrive/Person_Classification/Images/'
CATEGORIES = ['Female', 'Male']
IMG_SIZE = 224

In [4]:
data = []

for category in CATEGORIES:
    folder = os.path.join(DIRECTORY, category)
    label = CATEGORIES.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv2.imread(img_path)  #Converts to img array
        img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
        data.append([img_arr, label])
        # print(img_arr.shape)
        # plt.imshow(img_arr)
        # break


In [5]:
print(len(data))
random.shuffle(data)

800


In [6]:
X = []
y = []

for features, labels in data:
    X.append(features)
    y.append(labels)

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
X = X/255  #Feature Scaling

In [9]:
print(X.shape)
print(y.shape)
print(X.shape[1:])

(800, 224, 224, 3)
(800,)
(224, 224, 3)


In [10]:
model = Sequential()
model.add(Conv2D(128, (3,3), activation='relu', input_shape=X.shape[1:]))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu' ))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), activation='relu' ))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(.3))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(.3))
model.add(Dense(1, activation = 'sigmoid'))


In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# input_shape=X.shape
# model.build(input_shape)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

In [14]:
import tensorflow as tf
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [15]:
model_history = model.fit(X, y, epochs=20, validation_split=0.1, batch_size=16)

Epoch 1/20
45/45 [==============================] - 9s 62ms/step - loss: 0.7025 - accuracy: 0.4903 - val_loss: 0.6930 - val_accuracy: 0.5375
Epoch 2/20
45/45 [==============================] - 2s 54ms/step - loss: 0.6934 - accuracy: 0.5111 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 3/20
45/45 [==============================] - 3s 56ms/step - loss: 0.6841 - accuracy: 0.5889 - val_loss: 0.6896 - val_accuracy: 0.5375
Epoch 4/20
45/45 [==============================] - 3s 57ms/step - loss: 0.6068 - accuracy: 0.6750 - val_loss: 0.6404 - val_accuracy: 0.6625
Epoch 5/20
45/45 [==============================] - 2s 54ms/step - loss: 0.4783 - accuracy: 0.7764 - val_loss: 0.6828 - val_accuracy: 0.6250
Epoch 6/20
45/45 [==============================] - 2s 54ms/step - loss: 0.3047 - accuracy: 0.8764 - val_loss: 0.9462 - val_accuracy: 0.6625
Epoch 7/20
45/45 [==============================] - 3s 56ms/step - loss: 0.1997 - accuracy: 0.9181 - val_loss: 1.3695 - val_accuracy: 0.6375
Epoch 8/20
45

In [16]:
model.save('/content/drive/MyDrive/Person_Classification/model.h5')

# *Prediction:*


In [37]:
from PIL import Image
img = Image.open('/content/drive/MyDrive/Person_Classification/Images/Male/male_245.jpg')
img = img.resize((224,224))

In [38]:
img_array = np.array(img)
img_array.shape

(224, 224, 3)

In [39]:
img_array = np.expand_dims(img_array, axis=0)
img_array.shape

(1, 224, 224, 3)

In [40]:
result = model.predict(img_array)
result

1/1 [==============================] - 0s 27ms/step


array([[1.]], dtype=float32)

In [41]:
# argmax_index = np.argmax(result, axis=1)
if result[0][0] == 0:
    print("Predicted: Female")
else:
    print("Predicted: Male")

Predicted: Male
